## Lesson Notebook 6: Machine Translation

In this notebook we will look at several examples related to machine translation:

   * Simple translation examples with T5

   * Translation example with M2M100 - many more languages

   * MT metrics examples

   * Subword models and tokenizers



<a id = 'returnToTop'></a>

## Notebook Contents
  * 1. [Setup](#setup)
  * 2. [Simple Translation Model](#simpleTranslation)
  * 3. [M2M100 Translation Example](#m2mTranslation)
  * 4. [Machine Translation Metrics](#translationMetrics)  
  * 5. [Subword Models](#subwordModels)
  * [Answers](#answers)







  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2024-fall-main/blob/master/materials/lesson_notebooks/lesson_6_Machine_Translation.ipynb)

[Return to Top](#returnToTop)  
<a id = 'setup'></a>

## 1. Setup


We'll start with the usual setup. We need to begin with the sentencepiece code in order to tokenize the text for some of the models.

In [1]:
!pip install -q sentencepiece

In [2]:
#!pip install -q transformers
!pip install tensorflow==2.15.0 --quiet #15 13
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-datasets==4.8 --quiet #8
!pip install tensorflow-text==2.15.0 --quiet #15
!pip install transformers==4.17 --quiet #4.40.2 #4.37.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.65 requires ml-dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 50.3 MB/s eta 0:00:00


In [3]:
!pip install tensorflow -U --quiet #15 13
!pip install keras -U --quiet
!pip install tensorflow-datasets -U --quiet #8
!pip install tensorflow-text -U --quiet #15
!pip install transformers -U --quiet #4.40.2 #4.37.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.3 MB/s eta 0:00:00


In [4]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [5]:
!pip install -q git+https://github.com/keras-team/keras-nlp.git --upgrade

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
#Am I running a GPU and what type is it?
!nvidia-smi

Sun Sep 29 03:22:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

[Return to Top](#returnToTop)  
<a id = 'simpleTranslation'></a>


## 2. Simple Translation Example

These T5 models are trained to translate in one direction only.  For example, they can translate from English to French but not from French to English.

Let's test this out.

In [8]:
#import T5 and show
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration

In [9]:
SENTENCE_TO_TRANSLATE = ( "PG&E stated it scheduled the blackouts in response to forecasts for high winds \
            amid dry conditions.")

BACK_TRANSLATE_TEST = ("PG&E a déclaré qu'elle avait prévu les panne de courant.")

In [10]:
t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-base') #also t5-small and t5-large
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

t5_model.summary()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Add the prompt to the sentence we want to translate so the model knows what we want it to do with the input.

In [11]:
t5_input_text = "translate english to french: " + SENTENCE_TO_TRANSLATE

In [12]:
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')

**QUESTION 1**: What do the inputs look like?  We've already seen BERT inputs. What's happening with T5? What's the same as what we saw with BERT and what's different?

In [13]:
t5_inputs

{'input_ids': <tf.Tensor: shape=(1, 29), dtype=int32, numpy=
array([[13959, 22269,    12, 20609,    10,     3,  7861,   184,   427,
         4568,    34,  5018,     8,  1001,   670,     7,    16,  1773,
           12,  7555,     7,    21,   306, 13551, 18905,  2192,  1124,
            5,     1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 29), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [14]:
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                   max_length=20)

print([t5_tokenizer.decode(g, skip_special_tokens=True,
                           clean_up_tokenization_spaces=False) for g in t5_summary_ids])

["PG&E a déclaré qu'elle avait prévu les panne de courant"]


Not bad. Now let's try the reverse even though we know the model wasn't trained to translate in that direction.  What do you think it will do?

In [15]:
t5_back_text = "translate french to english: " + BACK_TRANSLATE_TEST

In [16]:
t5_binputs = t5_tokenizer([t5_back_text], return_tensors='tf')

The decoder still runs and emits language, specifically French, as requested.  These models will pretty much always produce some output but you need to make sure that you're asking it to do something it can and that it is doing the right thing.

In [17]:
t5_summary_ids = t5_model.generate(t5_binputs['input_ids'],
                                   max_length=20)

print([t5_tokenizer.decode(g, skip_special_tokens=True,
                           clean_up_tokenization_spaces=False) for g in t5_summary_ids])

["PG&E a déclaré qu'elle avait prévu les panne de courant"]


[Return to Top](#returnToTop)  
<a id = 'm2mTranslation'></a>


## 3. M2M100 Translation Example

M2M100 is a large model that was pre-trained on many languages simultaneoulsy.  You do need to give it some clues about what you are expecting when it translates.  Typically this takes the form of specifying the input and the output languages.  Let's look at the tokenizer first. How can it handle 100 different languages?

In [18]:
from transformers import M2M100Config, M2M100Tokenizer

tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="fr")

tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

['▁Don',
 "'",
 't',
 '▁you',
 '▁love',
 '▁',
 '🤗',
 '▁Transform',
 'ers',
 '?',
 '▁We',
 '▁sure',
 '▁do',
 '.']

Now let's try to translate.  We have two original sentences, one in English and one in Chinese that have roughly the same meaning.

In [19]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

en_text = "Do not meddle in the affairs of wizards, for they are subtle and quick to anger."
chinese_text = "不要插手巫師的事務, 因為他們是微妙的, 很快就會發怒."

tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="zh")
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [20]:
encoded_zh = tokenizer(chinese_text, return_tensors="pt")

Let's start by taking the Chinese sentence and translate it back in to English.

In [21]:
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Do not interfere with the matters of the witches, because they are delicate and will soon be angry.']

Interesting and subtly different from our English original. Now we'll try translating the English to Chinese and then we'll take that Chinese output and translate it back into English.  This should give us an idea of how well the model works.

In [22]:
encoded_en = tokenizer(en_text, return_tensors="pt")

In [23]:
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("zh"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['不要介入魔術師的事情,因為他們是微妙和快樂的憤怒。']

Now we'll store that Chinese output in a variable so we can translate back to English.

In [24]:
chinese_back_text = '不要介入魔術師的事情,因為他們是微妙和快樂的憤怒。'

In [25]:
encoded_zhb = tokenizer(chinese_back_text, return_tensors="pt")

In [26]:
generated_tokens = model.generate(**encoded_zhb, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Do not interfere with the things of the witches, because they are delicate and pleasant anger.']

Now you can see how far it has drifted as we have translated back and forth. With some care this approach can be used to generate novel content that can augment a training set (as long as the drift isn't too bad). This is what we call back translation.

[Return to Top](#returnToTop)  
<a id = 'translationMetrics'></a>

## 4. Machine Translation Metrics

HuggingFace provides a library called evaluate that includes a large number of metrics.  We'll use two of them here.

In [27]:
!pip install -q evaluate
import evaluate

from datasets import DownloadConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00


### 4.1 BLEU example

The [BLEU metric](https://huggingface.co/spaces/evaluate-metric/bleu) has been around for awhile. Let's run an example of the scoring using the function provided by the evaluate library from HuggingFace.

In [28]:
#let's manually create some candidates and references
#individual sentece example - this is best to experiment with.
bleu_candidate = ["the earth trembled in Japan again on Monday the 4th of September"
]

bleu_reference = [["earthquakes hit Japan again on Monday September 4"]
]

#multiple pairs of inputs and reference outputs
bleu_candidates = ["the earth trembled in Japan again on Monday the 4th of September",
                   "earthquakes struck Japan again on Monday the 4th of September"
]
bleu_references = [
                   ["earthquakes hit Japan again on Monday September 4"],
                   ["On September 4th , a Monday , Japan had another earthquake"]
]

Let's first try our individual candidate and reference.  They're both sort of saying the same thing.  Does the BLEU score reflect that similarity?

In [29]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=bleu_candidate, references=bleu_reference)
print(results)

{'bleu': 0.22416933501922287, 'precisions': [0.4166666666666667, 0.2727272727272727, 0.2, 0.1111111111111111], 'brevity_penalty': 1.0, 'length_ratio': 1.5, 'translation_length': 12, 'reference_length': 8}


BLEU is typically used in aggregate with multiple candidates as well as multiple reference examples for each sentence pair.  Here we run the candidates and references so you can see how it's done.

In [30]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=bleu_candidates, references=bleu_references)
print(results)

{'bleu': 0.14367696612929734, 'precisions': [0.4090909090909091, 0.15, 0.1111111111111111, 0.0625], 'brevity_penalty': 1.0, 'length_ratio': 1.1578947368421053, 'translation_length': 22, 'reference_length': 19}


[Return to Top](#returnToTop)  
<a id = 'subwordModels'></a>

## 5. Subword Models

Different pretrained models use different subword models.  Each subword model identifies a different set of "tokens" based on an efficient representation of words and parts of words in the pre-training corpus.  The model has an embedding for each one of the subwords in its vocabulary.

We do not typically interact directly with the subword models but rather do so indirectly through the Tokenizer object.

Let's try the BERT base cased tokenizer.  It uses a wordpiece subword model.

In [31]:
#wordpiece
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

print(f'The vocabulary size is {tokenizer.vocab_size}')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

The vocabulary size is 28996


In [32]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['Don',
 "'",
 't',
 'you',
 'love',
 '[UNK]',
 'Transformers',
 '?',
 'We',
 'sure',
 'do',
 '.']

This is the same tokenizer code but instead it is loaded with the multilingual model version. Note that it contains many more tokens than BERT base because it has to be able to deal with multiple kinds of symbols.

In [33]:
#wordpiece
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
print(f'The vocabulary size is {tokenizer.vocab_size}')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

The vocabulary size is 105879


In [34]:
tokenizer.tokenize("你不喜欢🤗变形金刚吗？ 我们肯定会。")

['你',
 '不',
 '喜',
 '欢',
 '[UNK]',
 '变',
 '形',
 '金',
 '刚',
 '吗',
 '？',
 '我',
 '们',
 '肯',
 '定',
 '会',
 '。']

Let's put that first English sentence through the multilingual tokenizer.  It produces the same subwords for English even though it can also handle other languages as shown by it's much lager vocabulary size.

In [35]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['don',
 "'",
 't',
 'you',
 'love',
 '[UNK]',
 'transformers',
 '?',
 'we',
 'sure',
 'do',
 '.']

T5 uses the sentencepiece subword model.  Here we'll use the tokenizer for the multilingual version of T5 called mt5.  Notice the vocabulary size.

In [36]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
print(f'The vocabulary size is {tokenizer.vocab_size}')

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

The vocabulary size is 250100


In [37]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['▁Don',
 "'",
 't',
 '▁you',
 '▁love',
 '▁',
 '🤗',
 '▁',
 'Transformers',
 '?',
 '▁We',
 '▁sure',
 '▁do',
 '.']

The sentencepiece subword model includes a marker to indicate if a subword is at the begining of a word and thus, in English, is preceeded by a space.  This means that with sentence piece it is possible to accurately reconstruct the sentence because we explicitly identify the word boundaries.


Finally, let's look at **GPT2** which uses the BytePair Encoding subword model.  Its output will be completely different.

In [38]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(f'The vocabulary size is {tokenizer.vocab_size}')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

The vocabulary size is 50257


In [39]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['Don',
 "'t",
 'Ġyou',
 'Ġlove',
 'ĠðŁ',
 '¤',
 'Ĺ',
 'ĠTransformers',
 '?',
 'ĠWe',
 'Ġsure',
 'Ġdo',
 '.']

[Return to Top](#returnToTop)  
<a id = 'answers'></a>

## ANSWERS

1.  The T5 model doesn't have the token type ids that BERT uses to identify different segments.

